In [ ]:
import yfinance as yf
import numpy as np 
import pandas as pd 
import requests 
import xlsxwriter 
import math 
from scipy import stats 

### Importing Data (list of static S&P500 stocks)

In [ ]:
stocks = pd.read_csv('sp_500_stocks.csv')

In [ ]:
print(stocks)

### Looping through one S&P500 companies for respective metrics

In [ ]:
data = yf.Ticker('AAPL').info
print(data)

In [ ]:
price = data['currentPrice']
trailing_PE = data['trailingPE']
forward_PE = data['forwardPE']
pb_ratio = data['priceToBook']
ps_ratio = data['priceToSalesTrailing12Months']
ev_to_ebitda = data['enterpriseToEbitda']

### Looping through all S&P500 companies for respective metrics

In [ ]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Forward Price-to-Earnings Ratio',
    'Forward PE Percentile',
    'Trailing Price-to-Earnings Ratio',
    'Trailing PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

rv_dataframe

In [ ]:
for symbol in stocks['Ticker']:
    try:    
        data = yf.Ticker(symbol).info
        new_row = pd.DataFrame([[symbol,
                data['currentPrice'],
                'N/A',
                data['forwardPE'],
                'N/A',
                data['trailingPE'],
                'N/A',
                data['priceToBook'],
                'N/A',
                data['priceToSalesTrailing12Months'],
                'N/A',
                data['enterpriseToEbitda'],
                'N/A',
                'N/A']], columns = rv_columns)
        rv_dataframe = pd.concat([rv_dataframe, new_row], ignore_index=True)
    except Exception as e:
        continue

rv_dataframe

### Checking for any rows with NA/NaN values

In [ ]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

### Determining percentile for each metric for each stock in dataframe

In [ ]:
metrics = {'Forward Price-to-Earnings Ratio': 'Forward PE Percentile',
           'Trailing Price-to-Earnings Ratio': 'Trailing PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile'}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(rv_dataframe[metric])

#Print the entire DataFrame    
rv_dataframe

### Calculating Robust Value (RV) for each company in the dataframe

In [ ]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

### Selecting 50 companies based on lowest RV Score

In [ ]:
rv_dataframe.sort_values(by = 'RV Score', inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)

rv_dataframe

### Determining Portfolio and Position Size

In [ ]:
while True:
    portfolio_size = input("Please enter the value of your portfolio: ")
    
    try:
        val = float(portfolio_size)
        break  # Exit the loop if the input is a valid float
    except ValueError:
        print("That's not a number! Please try again.")

In [ ]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
print("Position Size for each company is: $" f'{position_size}')

for i in range(0, len(rv_dataframe.index)):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])

rv_dataframe

### Formatting Excel Output

In [ ]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [ ]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Forward Price-to-Earnings Ratio', float_template],
                    'E': ['Forward PE Percentile', percent_template],
                    'F': ['Trailing Price-to-Earnings Ratio', float_template],
                    'G': ['Trailing PE Percentile', percent_template],
                    'H': ['Price-to-Book Ratio', float_template],
                    'I': ['PB Percentile',percent_template],
                    'J': ['Price-to-Sales Ratio', float_template],
                    'K': ['PS Percentile', percent_template],
                    'L': ['EV/EBITDA', float_template],
                    'M': ['EV/EBITDA Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

In [ ]:
writer.close()

### Backtesting Strategy (Not Possible)

In [ ]:
import backtrader as bt
from datetime import datetime, timedelta

In [ ]:
# Define the strategy class
class QuantValueStrategy(bt.Strategy):
    params = dict(rebalance_days=30)

    def __init__(self):
        self.counter = 0

    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print(f'{dt.isoformat()} {txt}')

    def next(self):
        # Increment counter each day
        self.counter += 1

        # Rebalance the portfolio every 30 days
        if self.counter % self.params.rebalance_days == 0:
            self.log("Rebalancing portfolio...")
            self.rebalance_portfolio()

    def rebalance_portfolio(self):
        # Clear all current positions
        for data in self.datas:
            position = self.getposition(data)
            if position.size != 0:
                self.close(data)

        # Get S&P 500 stock data using yfinance
        stocks = pd.read_csv('sp_500_stocks.csv')
        rv_columns = ['Ticker', 'Price', 'ForwardPE', 'TrailingPE', 'PriceToBook', 'PriceToSales', 'EnterpriseToEBITDA']
        rv_dataframe = pd.DataFrame(columns=rv_columns)

        for symbol in stocks['Ticker']:
            try:
                data = yf.Ticker(symbol).info
                new_row = pd.DataFrame([[symbol,
                                        data['currentPrice'],
                                        data.get('forwardPE', np.nan),
                                        data.get('trailingPE', np.nan),
                                        data.get('priceToBook', np.nan),
                                        data.get('priceToSalesTrailing12Months', np.nan),
                                        data.get('enterpriseToEbitda', np.nan)]],
                                      columns=rv_columns)
                rv_dataframe = pd.concat([rv_dataframe, new_row], ignore_index=True)
            except Exception as e:
                print(f"Error fetching data for {symbol}: {e}")
                continue

        # Remove rows with missing data
        rv_dataframe.dropna(inplace=True)

        # Calculate percentiles and RV score
        metrics = {
            'ForwardPE': 'ForwardPE Percentile',
            'TrailingPE': 'TrailingPE Percentile',
            'PriceToBook': 'PriceToBook Percentile',
            'PriceToSales': 'PriceToSales Percentile',
            'EnterpriseToEBITDA': 'EnterpriseToEBITDA Percentile'
        }

        for metric in metrics.keys():
            rv_dataframe[metrics[metric]] = rv_dataframe[metric].rank(pct=True)

        # Calculate the average of percentiles to determine RV Score
        rv_dataframe['RV Score'] = rv_dataframe[[metrics[m] for m in metrics]].mean(axis=1)

        # Select top 50 stocks based on RV Score
        rv_dataframe.sort_values(by='RV Score', inplace=True)
        rv_dataframe = rv_dataframe[:50]
        rv_dataframe.reset_index(drop=True, inplace=True)

        # Rebalance portfolio: equal weighting
        portfolio_value = self.broker.getvalue()
        position_size = portfolio_value / len(rv_dataframe)

        # Buy the top 50 stocks with equal weighting
        for i in range(0, len(rv_dataframe)):
            stock = rv_dataframe.loc[i, 'Ticker']
            price = rv_dataframe.loc[i, 'Price']

            if price > 0:
                data_feed = self.getdatabyname(stock)
                if data_feed is not None:
                    number_of_shares = int(position_size / price)

                    if number_of_shares > 0:
                        # Buy shares
                        self.buy(data=data_feed, size=number_of_shares)

In [ ]:
# Download historical data
def download_data(tickers, start_date, end_date):
    data = {}
    for ticker in tickers:
        try:
            df = yf.download(ticker, start=start_date, end=end_date)
            df['Open Interest'] = 0  # Required column for Backtrader
            data[ticker] = df
        except Exception as e:
            print(f"Error downloading {ticker}: {e}")
    return data

In [ ]:
cerebro = bt.Cerebro()

In [ ]:
# Get historical data for all S&P500 tickers over the last 10 years
start_date = (datetime.now() - timedelta(days=365 * 2)).strftime('%Y-%m-%d')
end_date = datetime.now().strftime('%Y-%m-%d')

In [ ]:
# Get tickers from the CSV file
stocks = pd.read_csv('sp_500_stocks.csv')
tickers = stocks['Ticker'].tolist()

In [ ]:
data = download_data(tickers, start_date, end_date)

In [ ]:
# Add data feeds to cerebro
for ticker, df in data.items():
    if not df.empty:
        data_feed = bt.feeds.PandasData(dataname=df)
        cerebro.adddata(data_feed, name=ticker)

In [ ]:
# Add the strategy
cerebro.addstrategy(QuantValueStrategy)

In [ ]:
# Set initial portfolio value
cerebro.broker.setcash(1000000)

In [ ]:
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trades')

In [ ]:
# Run the backtest
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
results = cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

In [ ]:
strat = results[0]
    
# Extract statistics from analyzers
sharpe = strat.analyzers.sharpe.get_analysis()
drawdown = strat.analyzers.drawdown.get_analysis()
returns = strat.analyzers.returns.get_analysis()
trades = strat.analyzers.trades.get_analysis()
    
# Print out or save the results
print(f'Sharpe Ratio: {sharpe["sharperatio"]}')
print(f'Max Drawdown: {drawdown["max"]["drawdown"]}%')
print(f'Annual Return: {returns["rnorm100"]}%')
    
# Accessing trade statistics
print(f'Total Trades: {trades.total.total}')
print(f'Winning Trades: {trades.won.total}')
print(f'Losing Trades: {trades.lost.total}')